# import

In [2]:
import warnings
warnings.simplefilter('ignore')

import os
import gc 
import sys
import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb 

from tqdm import tqdm
from collections import defaultdict
from scipy.spatial import distance
from multiprocessing import cpu_count

from sklearn.model_selection import StratifiedKFold, KFold 
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA, TruncatedSVD

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
pd.options.display.max_columns = None

NFOLDS = 6
SEED = 2019

# read

In [6]:
sorted(os.listdir('../feature'))

['ss_101.csv', 'ss_105.csv', 'ss_108.csv', 'ss_201.csv', 'ss_202.csv']

In [7]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [8]:
test.insert(1, 'target', 0)

In [11]:
var_cols = [f'var_{i}' for i in range(0, 200)]

In [12]:
ss_101 = pd.read_csv('../feature/ss_101.csv')

In [ ]:
cols = [f for f in ss_101]

In [13]:
ss_101.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,kmeans_2,kmeans_2_1_distance,kmeans_2_2_distance,kmeans_3,kmeans_3_1_distance,kmeans_3_2_distance,kmeans_3_3_distance,kmeans_4,kmeans_4_1_distance,kmeans_4_2_distance,kmeans_4_3_distance,kmeans_4_4_distance,kmeans_5,kmeans_5_1_distance,kmeans_5_2_distance,kmeans_5_3_distance,kmeans_5_4_distance,kmeans_5_5_distance,kmeans_6,kmeans_6_1_distance,kmeans_6_2_distance,kmeans_6_3_distance,kmeans_6_4_distance,kmeans_6_5_distance,kmeans_6_6_distance,kmeans_7,kmeans_7_1_distance,kmeans_7_2_distance,kmeans_7_3_distance,kmeans_7_4_distance,kmeans_7_5_distance,kmeans_7_6_distance,kmeans_7_7_distance,kmeans_8,kmeans_8_1_distance,kmeans_8_2_distance,kmeans_8_3_distance,kmeans_8_4_distance,kmeans_8_5_distance,kmeans_8_6_distance,kmeans_8_7_distance,kmeans_8_8_distance,kmeans_9,kmeans_9_1_distance,kmeans_9_2_distance,kmeans_9_3_distance,kmeans_9_4_distance,kmeans_9_5_distance,kmeans_9_6_distance,kmeans_9_7_distance,kmeans_9_8_distance,kmeans_9_9_distance
0,test_0,0.128580,2.325169,0.848823,1.287888,0.217705,0.340433,0.503601,0.506387,0.555762,1.006657,-0.437413,-0.184036,-0.280522,-1.767294,0.002152,-0.178331,-0.636761,-1.991131,0.041417,0.465690,-1.630150,0.190986,-0.952999,0.084145,-1.093209,1.585839,-0.921882,-0.693643,0.227132,-0.483567,0.739136,-0.524988,0.998684,-0.462166,-0.182081,0.829683,-0.724681,-1.037938,-0.202575,1.987908,-1.073613,1.614942,0.097712,0.860384,0.748175,-0.901576,0.303024,2.011251,0.220044,0.019716,0.451018,0.122059,-0.467509,1.710803,-0.197378,0.682818,0.756926,0.177326,-1.995002,0.468909,-0.054255,0.720462,-0.813118,0.065357,0.378351,0.504334,0.162672,0.755654,-1.189402,0.482595,0.744832,0.495208,-1.375551,1.426702,0.521288,0.799789,0.157048,-1.306747,-1.117209,2.304397,0.495811,0.946973,2.124585,1.088519,0.258075,0.530753,-0.107838,1.602077,1.781303,1.180855,0.667686,0.615405,0.595435,-0.767916,-1.836058,-0.793109,-1.013328,0.928904,0.522019,1.395744,-0.281291,0.789303,1.000680,-0.413429,0.688778,-0.880724,-0.174007,-0.766960,-1.218676,-0.774156,-0.454305,-0.249821,0.697997,1.352302,-1.007196,-0.250807,0.498041,-1.209127,-0.153947,1.300144,1.444800,0.059794,-0.142733,0.382467,-1.210443,0.004281,1.184604,1.014846,-1.022837,0.295146,0.339913,0.820050,1.923244,1.869645,0.103766,0.499130,-1.201971,0.013437,-1.700240,-1.341942,-1.749104,-0.745144,1.380443,0.253768,0.632322,-0.469404,0.527775,-1.132086,0.548332,0.001039,0.796029,-0.065230,-0.035879,-0.866173,2.417335,0.544385,-1.358209,0.056555,-0.225939,0.188639,-1.680658,-1.029156,-1.076678,-1.0016

# Preprocess

In [ ]:
df = pd.concat([train, test], axis=0)

In [ ]:
df_unique = df.nunique().reset_index().rename(columns={'0': 'count'})

In [ ]:
mms_df = df.copy()
ss_df = df.copy()

## MinMax

In [ ]:
mms = MinMaxScaler()
mms.fit(mms_df[var_columns])
mms_df[var_columns] = mms.transform(mms_df[var_columns])

## Standard

In [ ]:
ss = StandardScaler()
ss.fit(ss_df[var_columns])
ss_df[var_columns] = ss.transform(ss_df[var_columns])

## PCA

## TruncatedSVD

# prepare dataset

In [ ]:
train = df.iloc[:len(train_index)]
test = df.iloc[len(test_index):]

y = train['target']

not_use_cols = ['ID_code', 'target'] 
use_cols = [c for c in train.columns if c not in not_use_cols]

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
X = train[use_cols]
X_test = test[use_cols]

# pseudo labeling

In [ ]:
def cv_lightgbm(X, y, X_test, NFOLDS=5, SEED=6):
    params_in_train = {
        'num_boost_round': 20000,
        'early_stopping_rounds': 200,
        'verbose_eval': 500,
    }

    skf = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)

    oof = np.zeros(len(X))
    predictions = np.zeros(len(X_test))
    scores = {'train': [], 'valid': []}
    features = X.columns
    feature_importance_df = pd.DataFrame(columns=['fold', 'feature', 'importance'])

    for fold, (train_index, valid_index) in enumerate(skf.split(X, y)):
        print('fold: {}/{}'.format(fold+1, skf.n_splits))
        
        params = {
            'boosting': 'gbdt',
            'metric': 'auc',
            'objective': 'binary',
            'max_depth': -1,
            'num_leaves': 2,
            'min_data_in_leaf': 64,
            'bagging_freq': 5,
            'learning_rate': 0.01,
            'feature_fraction': 0.01,
            'bagging_fraction': 0.4,    
            'min_gain_to_split': 0.01,
            'min'
            'num_threads': cpu_count(),
            'verbose': -1,
            'seed': int(2**fold),
            'bagging_seed': int(2**fold),
            'drop_seed': int(2**fold),
        }
        
        dtrain = lgb.Dataset(X.iloc[train_index], label=y.iloc[train_index])
        dvalid = lgb.Dataset(X.iloc[valid_index], label=y.iloc[valid_index])
        
        model = lgb.train(params, dtrain, valid_sets=[dtrain, dvalid], **params_in_train)
        scores['train'].append(model.best_score['training']['auc'])
        scores['valid'].append(model.best_score['valid_1']['auc'])
        oof[valid_index] = model.predict(X.iloc[valid_index], num_iteration=model.best_iteration)

        fold_feature_importance_df = pd.DataFrame(columns=['fold', 'feature', 'importance'])
        fold_feature_importance_df['feature'] = features
        fold_feature_importance_df['importance'] = model.feature_importance()
        fold_feature_importance_df['fold'] = fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_feature_importance_df], axis=0)

        predictions += model.predict(X_test, num_iteration=model.best_iteration) / NFOLDS

        del model

    return oof, predictions, scores, feature_importance_df

In [ ]:
oof, predictions, scores, feature_importance_df = cv_lightgbm(X, y, X_test) 

In [ ]:
cv_score = roc_auc_score(y, oof)**0.5
print('Num folds: {}'.format(NFOLDS))
print('Train Scores: mean {:.5f}, max {:.5f}, min {:.5f}, std {:.5f}'.format(
    np.mean(scores['train']), np.max(scores['train']), np.min(scores['train']), np.std(scores['train'])))
print('Valid Scores: mean {:.5f}, max {:.5f}, min {:.5f}, std {:.5f}'.format(
    np.mean(scores['valid']), np.max(scores['valid']), np.min(scores['valid']), np.std(scores['valid'])))
print('CV Score: {:<8.5f}'.format(cv_score))

In [ ]:
y_test = np.array([np.where(x >= 0.5, 1, 0) for x in predictions])

In [ ]:
# y_test = np.array(predictions)

In [ ]:
y_test

In [ ]:
def cv_pseudo_labeling_lightgbm(X, y, X_test, y_test, NFOLDS=5, SEED=6):
    params_in_train = {
        'num_boost_round': 20000,
        'early_stopping_rounds': 200,
        'verbose_eval': 500,
    }

    skf = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)

    oof = np.zeros(len(X))
    predictions = np.zeros(len(X_test))
    scores = {'train': [], 'valid': []}
    features = X.columns
    feature_importance_df = pd.DataFrame(columns=['fold', 'feature', 'importance'])

    for fold, (train_index, valid_index) in enumerate(skf.split(X, y)):
        print('fold: {}/{}'.format(fold+1, skf.n_splits))
        
        params = {
            'boosting': 'gbdt',
            'metric': 'binary',
            'objective': 'binary',
            'max_depth': 6,
            'num_leaves': 12,
            'min_data_in_leaf': 64,
            'bagging_freq': 5,
            'learning_rate': 0.01,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.4,
            'reg_alpha': 2,
            'reg_lambda': 5,
            'min_gain_to_split': 0.01,
            'min_child_weight': 19,
            'num_threads': cpu_count(),
            'verbose': -1,
            'seed': int(2**fold),
            'bagging_seed': int(2**fold),
            'drop_seed': int(2**fold),
        }
        
        X_train = pd.concat([X.iloc[train_index], X_test], axis=0)
        y_train = np.concatenate([y.iloc[train_index], y_test], axis=0)
        
        dtrain = lgb.Dataset(X_train, label=y_train)
        dvalid = lgb.Dataset(X.iloc[valid_index], label=y.iloc[valid_index])
        
        model = lgb.train(params, dtrain, valid_sets=[dtrain, dvalid], **params_in_train)
        scores['train'].append(model.best_score['training']['binary_logloss'])
        scores['valid'].append(model.best_score['valid_1']['binary_logloss'])
        oof[valid_index] = model.predict(X.iloc[valid_index], num_iteration=model.best_iteration)

        fold_feature_importance_df = pd.DataFrame(columns=['fold', 'feature', 'importance'])
        fold_feature_importance_df['feature'] = features
        fold_feature_importance_df['importance'] = model.feature_importance()
        fold_feature_importance_df['fold'] = fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_feature_importance_df], axis=0)

        predictions += model.predict(X_test, num_iteration=model.best_iteration) / NFOLDS

        del model

    return oof, predictions, scores, feature_importance_df

In [ ]:
oof, psudeo_predictions, scores, feature_importance_df = cv_pseudo_labeling_lightgbm(X, y, X_test, y_test) 

In [ ]:
cv_score = roc_auc_score(y, oof)**0.5
print('Num folds: {}'.format(NFOLDS))
print('Train Scores: mean {:.5f}, max {:.5f}, min {:.5f}, std {:.5f}'.format(
    np.mean(scores['train']), np.max(scores['train']), np.min(scores['train']), np.std(scores['train'])))
print('Valid Scores: mean {:.5f}, max {:.5f}, min {:.5f}, std {:.5f}'.format(
    np.mean(scores['valid']), np.max(scores['valid']), np.min(scores['valid']), np.std(scores['valid'])))
print('CV Score: {:<8.5f}'.format(cv_score))

In [ ]:
submission = pd.read_csv(os.path.join('..', 'data', 'sample_submission.csv'))
submission['target'] = psudeo_predictions
submission.to_csv(os.path.join('..', 'submission', '{}_lightgbm.csv'.format(str(datetime.datetime.today().date()).replace('-', ''))), index=False)

In [ ]:
feature_importance_df['importance'] = feature_importance_df['importance'].astype('int')
ordered_feature = feature_importance_df.groupby(['feature'])['importance'].mean().sort_values(ascending=False).index
plt.figure(figsize=(12, 48))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
sns.barplot(x='importance', y='feature', data=feature_importance_df, order=ordered_feature)
plt.show()

In [ ]:
sns.distplot(submission['target'])
plt.show()

In [ ]:
submission = pd.read_csv(os.path.join('..', 'submission', '20190317_lightgbm.csv'))

In [ ]:
sns.distplot(submission['target'])
plt.show()